# Nettoyage et analyse du dataset

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
import plotly.graph_objects as go
import datetime
import matplotlib.pyplot as plt

##### Importation du csv dans un dataframe

In [3]:
df_ = pd.read_csv('./online_retail_II.csv')
df = df_.copy()

In [4]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


Attribute Information:
- InvoiceNo: Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation.
- StockCode: Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product.
- Description: Product (item) name. Nominal.
- Quantity: The quantities of each product (item) per transaction. Numeric.
- InvoiceDate: Invice date and time. Numeric. The day and time when a transaction was generated.
- Price: Unit price. Numeric. Product price per unit in sterling (Â£).
- CustomerID: Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer.
- Country: Country name. Nominal. The name of the country where a customer resides.

## Nettoyage des données

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB


Différentes remarques sur ces informations :
- La colonne 'InvoiceDate' est de type objet au lieu d'être en date.  
- La plupart des colonnes ont 1067371 lignes à l'exception de 'Description' et 'Customer ID', il y a donc des informations manquantes. La colonne 'Description' n'est pas vraiment utile, mais celle des 'Customer ID' étant primordiale nous allons supprimer les lignes où l'ID manque.  
- La colonne 'Customer ID' est de type float64 alors qu'un type int64 est certainement suffisant.  
- Il n'y a pas de colonne pour décrire le prix total en fonction de la quantité d'articles achetée, ce qui pourrait être une information intéressante.  

### Suppression des lignes identiques

Voyons maintenant s'il y a des doublons, s'il y en a nous les supprimerons

In [6]:
df[df.duplicated()].shape

(34335, 8)

In [7]:
df = df.drop_duplicates(keep='first')
df[df.duplicated()].shape

(0, 8)

### Recherche et suppression des valeurs manquantes (Customer ID)

In [8]:
nan_count = df.isna().sum()
print(nan_count.sort_values)

<bound method Series.sort_values of Invoice             0
StockCode           0
Description      4275
Quantity            0
InvoiceDate         0
Price               0
Customer ID    235151
Country             0
dtype: int64>


In [9]:
df = df[df['Customer ID'].notna()]

### Conversion des données

In [10]:
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

In [11]:
df['Customer ID'] = df['Customer ID'].astype(np.int64)

### Nettoyage tourné "métier"

In [12]:
df.describe()

,Quantity,Price,Customer ID
count,797885.000000,797885.000000,797885.000000
mean,12.602980,3.702732,15313.062777
std,191.670371,71.392549,1696.466663
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13964.000000
50%,5.000000,1.950000,15228.000000
75%,12.000000,3.750000,16788.000000
max,80995.000000,38970.000000,18287.000000


On va essayer d'identifier dans le dataset les lignes qui ne correspondent pas à des achats

- Les prix négatifs correspondent à des annulations de commande, identifiées par un C dans le numéro de facture. On conserve ces lignes

- Certains articles sont au prix de 0€, il s'agit sans doute de cadeau, cela ne donne donc pas d'informations sur le comportement **d'achat** des clients. On va supprimer ces lignes

In [13]:
df= df[df['Price']> 0]

- On va recherche si certaines lignes correspondent à des tests ou des frais de ports

In [14]:
shipping_fees_data = df[df['Description'].str.contains('shipping|fees', case=False)]

print(shipping_fees_data)


Empty DataFrame
Columns: [Invoice, StockCode, Description, Quantity, InvoiceDate, Price, Customer ID, Country]
Index: []


On a remarqué que certains stocks code ne contiennent que des lettres (hors un stock code est censé être composé de 5 chiffres). On va essayer de comprendre à quoi correspondent ces stock codes.

In [15]:
letters_only_stockcodes = df[df['StockCode'].str.isalpha()]['StockCode'].unique()
print(letters_only_stockcodes)


['POST' 'D' 'M' 'PADS' 'ADJUST' 'DOT' 'CRUK']


- POST : Postage, ce code correspond aux frais de port, on décide de supprimer ces lignes

In [16]:
# On va afficher les descriptions uniques associées au Stock code POST
filtered_df = df[df['StockCode'] == 'POST']
unique_descriptions = filtered_df['Description'].unique()
for description in unique_descriptions:
    print(description)


POSTAGE


- M : Manual, correspondant à des lignes entrées manuellement, on converserve ces lignes

In [17]:
# On va afficher les descriptions uniques associées au Stock code M
filtered_df = df[df['StockCode'] == 'M']
unique_descriptions = filtered_df['Description'].unique()
for description in unique_descriptions:
    print(description)


Manual


- D : Discount : correspond à des remises, on supprime également ces lignes

In [18]:
# On va afficher les descriptions uniques associées au Stock code D
filtered_df = df[df['StockCode'] == 'D']
unique_descriptions = filtered_df['Description'].unique()
for description in unique_descriptions:
    print(description)

Discount


- PADS : Pads to match all cushions, cela semble correspondre à un produit, (pads = coussins), on conserve ces lignes

In [19]:
# On va afficher les descriptions uniques associées au Stock code PADS
filtered_df = df[df['StockCode'] == 'PADS']
unique_descriptions = filtered_df['Description'].unique()
for description in unique_descriptions:
    print(description)


PADS TO MATCH ALL CUSHIONS


- ADJUST : Adjustment by john on 26/01/2010 16	, ce code semble utiliser pour enregistrer les ajustements manuels ou les corrections apportées aux données, tels que les modifications de prix, les remises, les retours ou les corrections d'erreurs dans les transactions. On décide de supprimer ces lignes car elles ne donnent pas d'informations sur le comportement du client.

In [20]:
filtered_df = df[df['StockCode'] == 'ADJUST']
filtered_df.head(5)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
70976,495733,ADJUST,Adjustment by john on 26/01/2010 16,1,2010-01-26 16:21:00,68.34,14911,EIRE
70977,495735,ADJUST,Adjustment by john on 26/01/2010 16,1,2010-01-26 16:22:00,201.56,12745,EIRE
70978,495734,ADJUST,Adjustment by john on 26/01/2010 16,1,2010-01-26 16:22:00,205.82,14911,EIRE
70979,C495737,ADJUST,Adjustment by john on 26/01/2010 16,-1,2010-01-26 16:23:00,10.50,16154,United Kingdom
70980,495736,ADJUST,Adjustment by john on 26/01/2010 16,1,2010-01-26 16:23:00,21.00,12606,Spain


- DOT : DOTCOM POSTAGE, cela doit correspondre à des frais de port à l'instar du code POST, on supprime ces lignes

- CRUK : CRUK Commission, on va supprimer ces lignes

In [21]:
# On va afficher les descriptions uniques associées au Stock code CRUK
unique_descriptions = filtered_df['Description'].unique()
for description in unique_descriptions:
    print(description)


Adjustment by john on 26/01/2010 16
Adjustment by john on 26/01/2010 17
Adjustment by Peter on 24/05/2010 1


### Feature Engineering

#### Feature : Total Price

Création d'une colonne 'Total' pour avoir le prix selon la quantité achetée

In [22]:
df['TotalPrice'] = df['Quantity'] * df['Price']
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.0


#### Feature : Cancelled

Création d'une colonne "cancelled" (1 ou 0) pour pouvoir compter le nombre de retours effectués par chaque client au moment du groupby de manière plus pratique qu'avec le C dans le code facture.

In [23]:
df["cancelled"] = np.where(df["Quantity"] < 0, 1, 0)

In [24]:
# Trier le DataFrame par la colonne "Quantity" de manière décroissante
df_sorted = df.sort_values(by="Quantity", ascending=True)

# Afficher le DataFrame trié
df_sorted.head(100)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice,cancelled
1065883,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2011-12-09 09:27:00,2.08,16446,United Kingdom,-168469.60,1
587085,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,2011-01-18 10:17:00,1.04,12346,United Kingdom,-77183.60,1
507225,C536757,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-9360,2010-12-02 14:23:00,0.03,15838,United Kingdom,-280.80,1
359669,C524235,21088,SET/6 FRUIT SALAD PAPER CUPS,-7128,2010-09-28 11:02:00,0.08,14277,France,-570.24,1
359670,C524235,21096,SET/6 FRUIT SALAD PAPER PLATES,-7008,2010-09-28 11:02:00,0.13,14277,France,-911.04,1
...,...,...,...,...,...,...,...,...,...,...
680600,C549968,22151,PLACE SETTING WHITE HEART,-432,2011-04-13 14:24:00,0.36,14064,United Kingdom,-155.52,1
103623,C499282,21166,COOK WITH WINE METAL SIGN,-432,2010-02-25 18:55:00,1.69,16684,United Kingdom,-730.08,1
814595,C562221,22151,PLACE SETTING WHITE HEART,-432,2011-08-03 14:22:00,0.36,16525,United Kingdom,-155.52,1
359649,C524235,84927D,LA PALMIERA TILE HOOK,-432,2010-09-28 11:02:00,0.08,14277,France,-34.56,1


### Identification des grossistes

On va essayer d'identifier la présence de grossistes, pour selon leur nombre, les supprimers (si trop peu de grossistes pour conduitre à une analyse) ou les regrouper dans un cluster puisqu'il s'agit d'une catégorie de client à part entière.

On recherche, par pays, les clients avec plus de 5000 produits par factures

In [25]:
# Groupby par pays, client et facture, et calculer la somme des quantités positives par facture
country_customer_invoice_quantities = df[df['Quantity'] > 0].groupby(['Country', 'Customer ID', 'Invoice'])['Quantity'].sum()

# Filtrer les données pour les clients ayant plus de x produits par facture
filtered_data = country_customer_invoice_quantities[country_customer_invoice_quantities > 5000]

# Réinitialiser l'index pour accéder aux colonnes "Country", "Customer ID" et "Invoice"
filtered_data = filtered_data.reset_index()

# Groupby par client et compter le nombre d'invoices différentes
customer_invoice_counts = filtered_data.groupby('Customer ID')['Invoice'].nunique()

# Groupby par pays et afficher les identifiants des clients ayant plus de 10000 produits par facture
potential_wholesalers = filtered_data.groupby('Country')['Customer ID'].unique()

# Afficher les potentiels grossistes par pays avec le nombre d'invoices différentes par client
for country, customers in potential_wholesalers.items():
    print(f"Pays : {country}")
    print("Identifiants des clients et nombre d'invoices différentes :")
    for customer in customers:
        invoice_count = customer_invoice_counts[customer]
        print(f"Customer ID : {customer}, Nombre d'invoices différentes : {invoice_count}")
    print()


Pays : Australia
Identifiants des clients et nombre d'invoices différentes :
Customer ID : 12415, Nombre d'invoices différentes : 7

Pays : Denmark
Identifiants des clients et nombre d'invoices différentes :
Customer ID : 13902, Nombre d'invoices différentes : 4

Pays : EIRE
Identifiants des clients et nombre d'invoices différentes :
Customer ID : 14156, Nombre d'invoices différentes : 9

Pays : France
Identifiants des clients et nombre d'invoices différentes :
Customer ID : 12678, Nombre d'invoices différentes : 1
Customer ID : 14277, Nombre d'invoices différentes : 1

Pays : Japan
Identifiants des clients et nombre d'invoices différentes :
Customer ID : 12754, Nombre d'invoices différentes : 1

Pays : Netherlands
Identifiants des clients et nombre d'invoices différentes :
Customer ID : 14646, Nombre d'invoices différentes : 24

Pays : Sweden
Identifiants des clients et nombre d'invoices différentes :
Customer ID : 17404, Nombre d'invoices différentes : 2

Pays : United Kingdom
Identi

On compte maintenant le nombre d'objets uniques commandés plus de x fois par client et par pays

In [26]:
# Seuil de quantité
threshold = 5000

# Filtrer les données pour les clients ayant commandé plus de x fois un même produit
filtered_data = df.groupby(['Country', 'Customer ID', 'StockCode'])['Quantity'].sum().reset_index()
filtered_data = filtered_data[filtered_data['Quantity'] > threshold]

# Compter le nombre d'objets uniques commandés plus de x fois par client et par pays
product_counts = filtered_data.groupby(['Country', 'Customer ID'])['StockCode'].nunique().reset_index()

# Filtrer les clients ayant au moins un produit différent commandé plus de x fois
potential_wholesalers = product_counts[product_counts['StockCode'] > 1]

# Afficher les potentiels grossistes par pays avec le nombre d'objets différents commandés plus de x fois
for country in potential_wholesalers['Country'].unique():
    print(f"Pays : {country}")
    print(f"Identifiants des clients et nombre d'objets différents commandés plus de {threshold} fois :")
    customers = potential_wholesalers[potential_wholesalers['Country'] == country]['Customer ID']
    for customer in customers:
        product_count = potential_wholesalers[(potential_wholesalers['Country'] == country) & (potential_wholesalers['Customer ID'] == customer)]['StockCode'].values[0]
        print(f"Customer ID : {customer}, Nombre d'objets différents commandés plus de {threshold} fois : {product_count}")
    print()


Pays : Denmark
Identifiants des clients et nombre d'objets différents commandés plus de 5000 fois :
Customer ID : 13902, Nombre d'objets différents commandés plus de 5000 fois : 13

Pays : Netherlands
Identifiants des clients et nombre d'objets différents commandés plus de 5000 fois :
Customer ID : 14646, Nombre d'objets différents commandés plus de 5000 fois : 6

Pays : United Kingdom
Identifiants des clients et nombre d'objets différents commandés plus de 5000 fois :
Customer ID : 12931, Nombre d'objets différents commandés plus de 5000 fois : 2
Customer ID : 13027, Nombre d'objets différents commandés plus de 5000 fois : 6
Customer ID : 13316, Nombre d'objets différents commandés plus de 5000 fois : 2
Customer ID : 13687, Nombre d'objets différents commandés plus de 5000 fois : 3
Customer ID : 15769, Nombre d'objets différents commandés plus de 5000 fois : 3
Customer ID : 15838, Nombre d'objets différents commandés plus de 5000 fois : 2
Customer ID : 16333, Nombre d'objets différent

On peut maintenant parmis les clients identifié, s'interesser aux produits achetés pour deduire s'il s'agit de grossiste ou non

Un exemple : 

In [27]:
pd.set_option('display.max_rows', None)

customer_id = 13027
customer_data = df[df['Customer ID'] == customer_id]
customer_data.head(20)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice,cancelled
62873,494930,84992,72 SWEETHEART FAIRY CAKE CASES,240,2010-01-20 10:41:00,0.4,13027,United Kingdom,96.0,0
62874,494930,84991,60 TEATIME FAIRY CAKE CASES,240,2010-01-20 10:41:00,0.4,13027,United Kingdom,96.0,0
62875,494930,21977,PACK OF 60 PINK PAISLEY CAKE CASES,240,2010-01-20 10:41:00,0.4,13027,United Kingdom,96.0,0
62876,494930,21975,PACK OF 60 DINOSAUR CAKE CASES,240,2010-01-20 10:41:00,0.4,13027,United Kingdom,96.0,0
62877,494930,21212,PACK OF 72 RETRO SPOT CAKE CASES,240,2010-01-20 10:41:00,0.4,13027,United Kingdom,96.0,0
65447,495233,21213,PACK OF 72 SKULL CAKE CASES,240,2010-01-22 09:20:00,0.4,13027,United Kingdom,96.0,0
88931,497802,21975,PACK OF 60 DINOSAUR CAKE CASES,240,2010-02-12 13:52:00,0.4,13027,United Kingdom,96.0,0
88932,497802,84992,72 SWEETHEART FAIRY CAKE CASES,240,2010-02-12 13:52:00,0.4,13027,United Kingdom,96.0,0
88933,497802,84991,60 TEATIME FAIRY CAKE CASES,240,2010-02-12 13:52:00,0.4,13027,United Kingdom,96.0,0
88934,497802,21977,PACK OF 60 PINK PAISLEY CAKE CASES,240,2010-02-12 13:52:00,0.4,13027,United Kingdom,96.0,0



D'après les informations fournies, le client qui a effectué ces achats semble être quelqu'un qui achète divers moules à gâteaux en grandes quantités. Ils ont acheté différents types de moules à gateau. Ces achats suggèrent que le client pourrait être impliqué dans des activités de pâtisserie ou de décoration de gâteaux.

Étant donné qu'ils ont effectué plusieurs achats de moules à gâteaux en grandes quantités, cela indique qu'ils pourraient exploiter une entreprise liée à la pâtisserie, à la décoration de gâteaux ou à la restauration. Le fait que les achats soient effectués en grandes quantités suggère également qu'ils pourraient avoir une forte demande pour ces produits, peut-être pour des événements, des fêtes ou des besoins commerciaux.

In [28]:
pd.set_option('display.max_rows', None)

customer_id = 14156
customer_data = df[df['Customer ID'] == customer_id]
customer_data.head(20)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice,cancelled
1000,489546,84050,PINK HEART SHAPE EGG FRYING PAN,6,2009-12-01 12:30:00,1.65,14156,EIRE,9.90,0
1001,489546,47568,ENGLISH ROSE DESIGN PEG BAG,6,2009-12-01 12:30:00,2.10,14156,EIRE,12.60,0
1002,489546,84782A,PINK 3 TIER GLASS PLATE STAND,2,2009-12-01 12:30:00,12.75,14156,EIRE,25.50,0
1003,489546,84782C,GREEN 3 TIER GLASS PLATE STAND,1,2009-12-01 12:30:00,12.75,14156,EIRE,12.75,0
1004,489546,21232,STRAWBERRY CERAMIC TRINKET BOX,12,2009-12-01 12:30:00,1.25,14156,EIRE,15.00,0
1005,489546,21231,SWEETHEART CERAMIC TRINKET BOX,12,2009-12-01 12:30:00,1.25,14156,EIRE,15.00,0
1006,489546,22186,RED STAR CARD HOLDER,6,2009-12-01 12:30:00,2.95,14156,EIRE,17.70,0
1007,489546,22169,FAMILY ALBUM WHITE PICTURE FRAME,4,2009-12-01 12:30:00,8.50,14156,EIRE,34.00,0
1008,489546,82494L,WOODEN FRAME ANTIQUE WHITE,6,2009-12-01 12:30:00,2.95,14156,EIRE,17.70,0
1009,489546,84933A,LARGE GREEN PEONY FOLDING STOOL,6,2009-12-01 12:30:00,2.10,14156,EIRE,12.60,0


Le client a effectué des achats en quantités variées, ce qui peut indiquer qu'ils achètent pour un usage personnel ou pour des besoins commerciaux tels que la revente ou l'exploitation d'un commerce de détail.

Il est difficile de différentier les grossistes, des clients qui organisent des évènenements par exemple. On conserve donc pour l'instant ces données.

### Sauvegarde du dataframe (pour l'utiliser pour faire l'analyse RFM et clustering)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 797815 entries, 0 to 1067370
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      797815 non-null  object        
 1   StockCode    797815 non-null  object        
 2   Description  797815 non-null  object        
 3   Quantity     797815 non-null  int64         
 4   InvoiceDate  797815 non-null  datetime64[ns]
 5   Price        797815 non-null  float64       
 6   Customer ID  797815 non-null  int64         
 7   Country      797815 non-null  object        
 8   TotalPrice   797815 non-null  float64       
 9   cancelled    797815 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 67.0+ MB


In [30]:
df.to_csv('dataset/clean_online_retail.csv', index=False)

In [31]:
df.head(10)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice,cancelled
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.4,0
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0,0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0,0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,100.8,0
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.0,0
5,489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01 07:45:00,1.65,13085,United Kingdom,39.6,0
6,489434,21871,SAVE THE PLANET MUG,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.0,0
7,489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,2009-12-01 07:45:00,5.95,13085,United Kingdom,59.5,0
8,489435,22350,CAT BOWL,12,2009-12-01 07:46:00,2.55,13085,United Kingdom,30.6,0
9,489435,22349,"DOG BOWL , CHASING BALL DESIGN",12,2009-12-01 07:46:00,3.75,13085,United Kingdom,45.0,0
